Imports

In [2]:
import tensorflow as tf
import numpy as np

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Activation, Flatten

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

Get the data.

The data is from https://keras.io/api/datasets/cifar10/.
It is a dataset of 50,000 32x32 color training images and 10,000 test images, labeled over 10 categories.

In [3]:
dataset = keras.datasets.cifar10

Load and process the data

In [13]:
(training_images, training_labels), (test_images, test_labels) = dataset.load_data()

training_images = training_images.reshape(50000, 32, 32, 3)
test_images = test_images.reshape(10000, 32, 32, 3)

Creating the model

In [14]:
model = Sequential([
    Conv2D(64, (3, 3), padding='same', input_shape=training_images.shape[1:], activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3)),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")

Fitting the model using 15 epochs

In [ ]:
model.fit(training_images, training_labels, epochs=15)

Epoch 1/15
50000/50000 [==============================] - 154s 3ms/step - loss: 2.1221

In [ ]:
model.evaluate(test_images, test_labels)